In [10]:
import pandas as pd
import numpy as np
import os.path as op
import matplotlib.pyplot as plt

In [11]:
from ta_lib.core.api import create_context, list_datasets, load_dataset, display_as_tabs,merge_info

In [12]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [13]:
google_df = load_dataset(context, 'cleaned/google')
product_df = load_dataset(context, 'cleaned/product')
sales_df = load_dataset(context, 'cleaned/sales')
media_df = load_dataset(context, 'cleaned/media')
theme_list_df = load_dataset(context, 'cleaned/theme_list')
tpl_df = load_dataset(context, 'cleaned/tpl')

### Emerging Theme in the google search data

In [ ]:
google_df['week_day'] = pd.to_datetime(google_df['date']).dt.dayofweek
google_df['week'] = pd.to_datetime(google_df['date']).dt.week
google_df['month'] = pd.to_datetime(google_df['date']).dt.month
google_df['quarter'] = pd.to_datetime(google_df['date']).dt.quarter
google_df['year'] = pd.to_datetime(google_df['date']).dt.year

In [6]:
g_df = google_df.drop(columns=['date','week_number','year_new'])
g_df = pd.merge(g_df,theme_list_df,on= 'claim_id', how = 'inner')
g_df = g_df.drop(columns='claim_id')
g_df

,platform,search_volume,week_day,week,month,quarter,year,claim_name
0,google,349,6,1,1,1,2014,gluten free
1,google,349,0,2,1,1,2014,gluten free
2,google,697,1,2,1,1,2014,gluten free
3,google,349,4,2,1,1,2014,gluten free
4,google,697,0,4,1,1,2014,gluten free
...,...,...,...,...,...,...,...,...
181560,google,42,2,30,7,3,2019,snickerdoodle
181561,google,84,2,39,9,3,2019,snickerdoodle
181562,amazon,135,0,23,6,2,2018,hemp seeds
181563,amazon,39,3,5,1,1,2019,gingerbread


In [7]:
g_df.claim_name.nunique()

160

In [8]:
d = g_df.groupby(['year','claim_name'])['search_volume'].sum().reset_index()
# d.sort_values(by= ['search_volume'], ascending=[False],inplace=True)

In [9]:
df = d.groupby(['year','claim_name'])['search_volume'].sum()

In [10]:
df.sort_values( ascending=False,inplace=True)
df = df.sort_index(level = 0, sort_remaining=False).to_frame().reset_index()

In [11]:
# year = df.year.unique()
# for i in year:
#     print(df[df.year == i].iloc[:5])

In [12]:
print(df[df.year == 2019].iloc[:5].reset_index(drop=True))

   year             claim_name  search_volume
0  2019  ethical - environment       14703112
1  2019                  honey       13585784
2  2019                 shrimp       12736320
3  2019             sugar free       10484779
4  2019       health (passive)       10377761


### Feature Table

In [14]:
import ta_lib.eda.api as eda

In [15]:
# google_df = google_df.drop(columns=['week_number','year_new'])

In [16]:
sum1 = eda.get_variable_summary(google_df)
sum2 = eda.get_variable_summary(product_df)
sum3 = eda.get_variable_summary(sales_df)
sum4 = eda.get_variable_summary(media_df)
sum5 = eda.get_variable_summary(theme_list_df)
sum6 = eda.get_variable_summary(tpl_df)

display_as_tabs([('google', sum1), ('product', sum2), ('sales', sum3)
                , ('media', sum4), ('theme_list', sum5), ('theme_production_list', sum6)])

Tabs
    [0] DataFrame(name='google', value=               Variable Na...)
    [1] DataFrame(name='product', value=           Variable Name D...)
    [2] DataFrame(name='sales', value=                          ...)
    [3] DataFrame(name='media', value=           Variable Name D...)
    [4] DataFrame(name='theme_list', value=           Variable Name D...)
    [5] DataFrame(name='theme_production_list', value=           Variable Name D...)

In [23]:
# GOOGLE
google_df['week'] = pd.to_datetime(google_df['date']).dt.week
google_df['month'] = pd.to_datetime(google_df['date']).dt.month
google_df['quarter'] = pd.to_datetime(google_df['date']).dt.quarter
google_df['year'] = pd.to_datetime(google_df['date']).dt.year

# SALES
sales_df['week'] = pd.to_datetime(sales_df['system_calendar_key_n']).dt.week
sales_df['month'] = pd.to_datetime(sales_df['system_calendar_key_n']).dt.month
sales_df['quarter'] = pd.to_datetime(sales_df['system_calendar_key_n']).dt.quarter
sales_df['year'] = pd.to_datetime(sales_df['system_calendar_key_n']).dt.year

# MEDIA
media_df['week'] = pd.to_datetime(media_df['published_date']).dt.week
media_df['month'] = pd.to_datetime(media_df['published_date']).dt.month
media_df['quarter'] = pd.to_datetime(media_df['published_date']).dt.quarter
media_df['year'] = pd.to_datetime(media_df['published_date']).dt.year

In [27]:
google_df.drop(columns=['date'],inplace=True)
sales_df.drop(columns=['system_calendar_key_n'],inplace=True)
media_df.drop(columns=['published_date'],inplace=True)

In [29]:
fs_df = pd.merge(sales_df, product_df, on='product_id', how= 'inner')
merge_info(sales_df,product_df, fs_df)

,n_cols,n_rows
left_df,8,4526182
right_df,2,67175
merged_df,9,4526182


In [17]:
theme_df = pd.merge(tpl_df, theme_list_df, on='claim_id', how= 'left')
merge_info(theme_list_df,tpl_df, theme_df)

,n_cols,n_rows
left_df,2,208
right_df,2,91485
merged_df,3,91485


In [19]:
theme_product_df = pd.merge(theme_df, product_df, on='product_id', how= 'left')
merge_info(theme_df, product_df, theme_product_df)

,n_cols,n_rows
left_df,3,91485
right_df,2,67175
merged_df,4,91485


In [23]:
theme_product_df.head()

,product_id,claim_id,claim_name,vendor
0,26,8,low carb,Others
1,29,8,low carb,Others
2,48,81,gmo free,Others
3,50,81,gmo free,Others
4,74,227,salmon,Others


In [28]:
gf_df = google_df.groupby(
        ['year', 'quarter', 'month', 'week', 'claim_id','platform']
        )['search_volume'].sum().to_frame().reset_index()
gf_df.head()

,year,quarter,month,week,claim_id,platform,search_volume
0,2014,1,1,1,8,google,30266
1,2014,1,1,1,39,google,272
2,2014,1,1,1,75,google,4448
3,2014,1,1,1,81,google,5207
4,2014,1,1,1,100,google,4613


In [46]:
media_df.rename(columns={'theme_id': 'claim_id'},inplace=True)

In [29]:
mf_df = media_df.groupby(
        ['year', 'quarter', 'month', 'week', 'claim_id']
        )['total_post'].sum().to_frame().reset_index()
mf_df.head()

,year,quarter,month,week,claim_id,total_post
0,2015,2,5,21,8.0,412
1,2015,2,5,21,15.0,4
2,2015,2,5,21,26.0,0
3,2015,2,5,21,38.0,4
4,2015,2,5,21,39.0,10


In [33]:
social_df = gf_df.merge(mf_df, on=['year', 'quarter', 'month', 'week', 'claim_id'], how='inner')
merge_info(gf_df,mf_df,social_df)

,n_cols,n_rows
left_df,7,44713
right_df,6,53421
merged_df,8,37622


In [34]:
fs_df = pd.merge(sales_df, theme_product_df, on='product_id', how= 'left')
merge_info(sales_df,theme_product_df, fs_df)

,n_cols,n_rows
left_df,8,4526182
right_df,4,91485
merged_df,11,7767420


In [41]:
final_df = fs_df.merge(social_df, on=['year', 'quarter', 'month', 'week', 'claim_id'], how='inner')
merge_info(fs_df,social_df,final_df)

,n_cols,n_rows
left_df,11,7767420
right_df,8,37622
merged_df,14,6208890


In [40]:
final_df

,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,week,month,quarter,year,claim_id,claim_name,vendor,platform,search_volume,total_post
0,37,1101.0,159,105,1,1,1,2016,158,chicken,Others,google,123790,325
1,34097,1768.0,364,182,1,1,1,2016,158,chicken,Others,google,123790,325
2,34103,12.0,2,0,1,1,1,2016,158,chicken,Others,google,123790,325
3,34142,3254.0,108,650,1,1,1,2016,158,chicken,Others,google,123790,325
4,34143,3162.0,109,656,1,1,1,2016,158,chicken,Others,google,123790,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208885,41063,5821.0,192,86,43,10,4,2018,191,bone health,Others,chewy,256,5662
6208886,41063,5821.0,192,86,43,10,4,2018,191,bone health,Others,google,489,5662
6208887,41065,5004.0,202,56,43,10,4,2018,191,bone health,Others,amazon,4622,5662
6208888,41065,5004.0,202,56,43,10,4,2018,191,bone health,Others,chewy,256,5662
